## Import libraries

In [1]:
import time
from datetime import date, datetime, timedelta
import requests
from bs4 import BeautifulSoup as BS

import csv
import pandas as pd
import numpy as np

import json

In [8]:
import sqlite3

In [37]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait

In [29]:
import textdistance

In [203]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import preprocessing

## Web scraping

### Setting up Selenium

In [38]:
driver = webdriver.Chrome()

In [39]:
driver.get(url)

In [128]:
url = 'https://www.petfinder.com/dog-breeds/affenpinscher/'
print(url)
driver = webdriver.Chrome()
driver.get(url_dog)
elems = driver.find_elements_by_class_name('grid.grid_gutterLg.m-grid_alignBottom')

https://www.petfinder.com/dog-breeds/affenpinscher/


In [134]:
driver.find_element_by_xpath('//*[@id="Site"]/div/div/div[1]/div/div[1]/div/div[1]/div[3]/div[2]/div[2]').text

'ENERGY LEVEL\n3 out of 5\nEXERCISE REQUIREMENTS\n3 out of 5\nPLAYFULNESS\n4 out of 5\nAFFECTION LEVEL\n5 out of 5\nFRIENDLINESS TO DOGS\n5 out of 5\nFRIENDLINESS TO OTHER PETS\n5 out of 5\nFRIENDLINESS TO STRANGERS\n4 out of 5\nWATCHFULNESS\n1 out of 5\nEASE OF TRAINING\n4 out of 5\nGROOMING REQUIREMENTS\n4 out of 5\nHEAT SENSITIVITY\n3 out of 5\nVOCALITY\n4 out of 5'

In [144]:
driver.quit()

### Pagination

#### Finding class with dog breed names and direct links to their website

In [ ]:
list_of_dogs = []
list_of_dogs_mod =[]
driver = webdriver.Chrome()
for i in range(1,26):
    print(i)
    all_dogs = f'https://www.petfinder.com/dog-breeds/?page={i}'
    driver.get(all_dogs)
    wait = WebDriverWait(driver, 3)
    elems = driver.find_elements_by_class_name('contentCard.contentCard_breed.m-contentCard_persistant')
    for x in elems:
        if len(x.text) > 3:
            dog = x.text.split('\n')[0]
            list_of_dogs.append(dog)
            list_of_dogs_mod.append(dog.lower().replace(' ','-'))
driver.quit()

#### Getting information about dog characteristics for each breed available on the website

In [ ]:
driver = webdriver.Chrome()
features = []
for dog in list_of_dogs_mod:
    url_dog = f"https://www.petfinder.com/dog-breeds/{dog}"
    print(url_dog)
    wait = WebDriverWait(driver, 5)
    try:
        driver.get(url_dog)
        features.append(driver.find_element_by_xpath('//*[@id="Site"]/div/div/div[1]/div/div[1]/div/div[1]/div[3]/div[2]/div[2]').text)
    except:
        print(dog, 'no website')
        features.append(None)
        continue
driver.quit()

In [161]:
len(features)

195

Found 195 different dog breeds on PetFinder website

In [250]:
first = pd.concat([pd.DataFrame([[col][0]], columns = ['label_name', 'energy_level']) for col in list(zip(list_of_dogs_mod,features))],ignore_index = True)

In [251]:
first.head()

,label_name,energy_level
0,affenpinscher,ENERGY LEVEL\n4 out of 5\nEXERCISE REQUIREMENT...
1,afghan-hound,ENERGY LEVEL\n2 out of 5\nEXERCISE REQUIREMENT...
2,airedale-terrier,ENERGY LEVEL\n4 out of 5\nEXERCISE REQUIREMENT...
3,akita,ENERGY LEVEL\n3 out of 5\nEXERCISE REQUIREMENT...
4,alaskan-klee-kai,ENERGY LEVEL\n4 out of 5\nEXERCISE REQUIREMENT...


#### Cleaning the scraped information

In [252]:
first['energy_level_2'] = first.energy_level.apply(lambda x: x.split('\n')[1].split(' ')[0] if x else x)
first['exercise_reqs'] = first.energy_level.apply(lambda x: x.split('\n')[3].split(' ')[0] if x else x)
first['playfulness'] = first.energy_level.apply(lambda x: x.split('\n')[5].split(' ')[0] if x else x)
first['affection_lvl'] = first.energy_level.apply(lambda x: x.split('\n')[7].split(' ')[0] if x else x)
first['friend_dogs'] = first.energy_level.apply(lambda x: x.split('\n')[9].split(' ')[0] if x else x)
first['friend_pets'] = first.energy_level.apply(lambda x: x.split('\n')[11].split(' ')[0] if x else x)
first['friend_strangers'] = first.energy_level.apply(lambda x: x.split('\n')[13].split(' ')[0] if x else x)
first['watchfulness'] = first.energy_level.apply(lambda x: x.split('\n')[15].split(' ')[0] if x else x)
first['easy_training'] = first.energy_level.apply(lambda x: x.split('\n')[17].split(' ')[0] if x else x)
first['grooming'] = first.energy_level.apply(lambda x: x.split('\n')[19].split(' ')[0] if x else x)
first['heat_sensitive'] = first.energy_level.apply(lambda x: x.split('\n')[21].split(' ')[0] if x else x)
first['vocality'] = first.energy_level.apply(lambda x: x.split('\n')[23].split(' ')[0] if x else x)

In [254]:
first.to_csv('dog_features_table.csv', index = False)

In [115]:
dog_features = pd.read_csv('dog_features_table.csv')

In [116]:
dog_features.head()

,label_name,energy_level,energy_level_2,exercise_reqs,playfulness,affection_lvl,friend_dogs,friend_pets,friend_strangers,watchfulness,easy_training,grooming,heat_sensitive,vocality
0,affenpinscher,ENERGY LEVEL\r\n4 out of 5\r\nEXERCISE REQUIRE...,4.0,3.0,4.0,4.0,3.0,3.0,3.0,1.0,3.0,4.0,4.0,5.0
1,afghan-hound,ENERGY LEVEL\r\n2 out of 5\r\nEXERCISE REQUIRE...,2.0,3.0,3.0,1.0,3.0,3.0,1.0,1.0,2.0,4.0,3.0,3.0
2,airedale-terrier,ENERGY LEVEL\r\n4 out of 5\r\nEXERCISE REQUIRE...,4.0,3.0,4.0,3.0,1.0,2.0,3.0,4.0,3.0,4.0,3.0,4.0
3,akita,ENERGY LEVEL\r\n3 out of 5\r\nEXERCISE REQUIRE...,3.0,3.0,3.0,3.0,1.0,2.0,1.0,4.0,3.0,2.0,5.0,5.0
4,alaskan-klee-kai,ENERGY LEVEL\r\n4 out of 5\r\nEXERCISE REQUIRE...,4.0,4.0,4.0,4.0,2.0,1.0,2.0,4.0,4.0,3.0,3.0,3.0


## Dog similarity

#### Image recogniition system for dog breeds and dog characteristics datasets will be merged on breed names to build a recommendation system for similar dog based on its features.

Reading labels from dog breed recognition model

In [117]:
classes_labels = pd.read_csv('label_list_trained.csv')
classes_labels.head()

,label_name
0,Afghan_hound
1,African_hunting_dog
2,Airedale
3,American_Staffordshire_terrier
4,Appenzeller


Modify labels to similar format as in PetFinder urls

In [118]:
classes_labels['label_name_mod'] = classes_labels.label_name.apply(lambda x: x.lower().replace('_', '-'))
classes_labels.head()

,label_name,label_name_mod
0,Afghan_hound,afghan-hound
1,African_hunting_dog,african-hunting-dog
2,Airedale,airedale
3,American_Staffordshire_terrier,american-staffordshire-terrier
4,Appenzeller,appenzeller


First try matching labels - by same name format

In [119]:
classes_labels['pet_finder_name'] = None
classes_labels.head()

,label_name,label_name_mod,pet_finder_name
0,Afghan_hound,afghan-hound,None
1,African_hunting_dog,african-hunting-dog,None
2,Airedale,airedale,None
3,American_Staffordshire_terrier,american-staffordshire-terrier,None
4,Appenzeller,appenzeller,None


In [120]:
classes_labels['pet_finder_name'][0]

In [121]:
common = []
for i, x in enumerate(list(classes_labels.label_name_mod)):
    if x in list(dog_features.label_name):
        classes_labels['pet_finder_name'][i] = x
        common.append(x)

In [122]:
len(common)

73

In [123]:
classes_labels.head()

,label_name,label_name_mod,pet_finder_name
0,Afghan_hound,afghan-hound,afghan-hound
1,African_hunting_dog,african-hunting-dog,None
2,Airedale,airedale,None
3,American_Staffordshire_terrier,american-staffordshire-terrier,american-staffordshire-terrier
4,Appenzeller,appenzeller,None


Only 73 out of 120 breeds were matched based on simple name modification. I will try to use one of text distance algorithms for better matching

In [126]:
most_similar = list(np.zeros(120))
most_similar_scores = []
for i, x in enumerate(list(classes_labels.label_name_mod)):
    most_similar[i] = '!'
    current = 0
    for j, y in enumerate(list(dog_features.label_name)):
        if current < textdistance.jaro_winkler(x,y):
            most_similar[i] = y
            current = textdistance.jaro_winkler(x,y)
    if current < 0.8:
        most_similar[i] = None
    most_similar_scores.append(current)

In [127]:
classes_labels['jaro-winkler'] = most_similar

In [128]:
classes_labels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 4 columns):
label_name         120 non-null object
label_name_mod     120 non-null object
pet_finder_name    73 non-null object
jaro-winkler       105 non-null object
dtypes: object(4)
memory usage: 3.8+ KB


Jaro-Winkler text distance algorithm resulted in 105 matching dog names. I will go on with this value to build a recommender system.

Finding dog features based on the breed name (format as in images)

In [130]:
label = 'Airedale'
find_label = classes_labels.loc[classes_labels['label_name'] == label, 'jaro-winkler'].values[0]
dog_features.loc[dog_features['label_name'] == find_label].values[0][2:]

array([4.0, 3.0, 4.0, 3.0, 1.0, 2.0, 3.0, 4.0, 3.0, 4.0, 3.0, 4.0],
      dtype=object)

#### Build a dataframe with image format labels and matching dog characteristics

In [ ]:
features = []
for label in list(classes_labels['label_name']):
    find_label = classes_labels.loc[classes_labels['label_name'] == label, 'jaro-winkler'].values[0]
    if find_label != None:
        features.append(list(dog_features.loc[dog_features['label_name'] == find_label].values[0][2:]))
        print(features)
    else:
        features.append(np.zeros(0))
        print(np.zeros(11))

In [151]:
comb = pd.DataFrame(features,columns=dog_features.columns[2:])

In [157]:
comb['label_name'] = list(classes_labels['label_name'])
comb['jaro_winkler'] = most_similar

In [158]:
comb = comb.set_index('label_name').reset_index()

In [163]:
comb.to_csv('combined_features.csv',index = False)

In [170]:
# rest of rows updated by hand
comb_updated = pd.read_csv('comined_features_updated.csv').fillna('None')

In [214]:
def find_features(label):
    feature_names = comb_updated.columns[1:-1]
    find_features = comb_updated.loc[comb_updated['label_name'] == label].values[0][1:-1]
    for i, feature in enumerate(feature_names):
        print(f'\n {feature}: {find_features[i]}')

In [215]:
label = 'Boston_bull'
find_features(label)


 energy_level_2: 3

 exercise_reqs: 1

 playfulness: 3

 affection_lvl: 3

 friend_dogs: 3

 friend_pets: 4

 friend_strangers: 3

 watchfulness: 1

 easy_training: 3

 grooming: 1

 heat_sensitive: 5

 vocality: 5


Scaling dog characteristics

In [248]:
x = comb_updated[comb_updated.columns[1:-1]].values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
comb_scaled = pd.DataFrame(x_scaled)

In [251]:
comb_scaled.shape

(120, 12)

Finding similar dog based on the dog characteristics - example dog - Border terrier - index in images - 10

In [249]:
for dog in [x[0] for x in sorted(list(zip(list(comb_updated.label_name),(cosine_similarity(comb_scaled)[10]))), key=lambda x: x[1], reverse = True)[:6]]:
    print('\n\n',dog)
    find_features(dog)



 Border_terrier

 energy_level_2: 3

 exercise_reqs: 3

 playfulness: 3

 affection_lvl: 3

 friend_dogs: 3

 friend_pets: 2

 friend_strangers: 3

 watchfulness: 1

 easy_training: 4

 grooming: 3

 heat_sensitive: 3

 vocality: 4


 miniature_poodle

 energy_level_2: 4

 exercise_reqs: 3

 playfulness: 5

 affection_lvl: 4

 friend_dogs: 4

 friend_pets: 3

 friend_strangers: 4

 watchfulness: 1

 easy_training: 5

 grooming: 5

 heat_sensitive: 3

 vocality: 5


 Norwich_terrier

 energy_level_2: 4

 exercise_reqs: 3

 playfulness: 3

 affection_lvl: 4

 friend_dogs: 3

 friend_pets: 3

 friend_strangers: 3

 watchfulness: 1

 easy_training: 3

 grooming: 3

 heat_sensitive: 3

 vocality: 5


 miniature_schnauzer

 energy_level_2: 3

 exercise_reqs: 3

 playfulness: 4

 affection_lvl: 4

 friend_dogs: 3

 friend_pets: 3

 friend_strangers: 3

 watchfulness: 1

 easy_training: 3

 grooming: 3

 heat_sensitive: 3

 vocality: 5


 West_Highland_white_terrier

 energy_level_2: 4

 exe

This approach will be used in dog recognition and recommendation app in 'streamlit_script.py and prediction.py'